## 1. Definición de la Clase Vector2D: Fundamentos Matemáticos del Espacio Vectorial

En esta celda definimos la estructura base de un **vector bidimensional** sobre el campo de los números reales $\mathbb{R}^2$. Un vector $\vec{v} \in \mathbb{R}^2$ se representa como un par ordenado:

$$\vec{v} = (x, y) \quad \text{donde} \quad x, y \in \mathbb{R}$$

### Propiedades Algebraicas

El conjunto $\mathbb{R}^2$ con las operaciones de **suma vectorial** y **producto por escalar** forma un **espacio vectorial** que satisface los axiomas de:

- **Cerradura**: $\vec{u} + \vec{v} \in \mathbb{R}^2$
- **Conmutatividad**: $\vec{u} + \vec{v} = \vec{v} + \vec{u}$
- **Asociatividad**: $(\vec{u} + \vec{v}) + \vec{w} = \vec{u} + (\vec{v} + \vec{w})$
- **Elemento neutro**: $\exists \vec{0} = (0,0) \mid \vec{v} + \vec{0} = \vec{v}$
- **Inverso aditivo**: $\exists -\vec{v} \mid \vec{v} + (-\vec{v}) = \vec{0}$

### Implementación con p5.Vector

Utilizamos el patrón **Wrapper** (envoltorio) para encapsular la clase `p5.Vector` nativa de p5.js, permitiendo:

1. **Compatibilidad**: Mantener la API de tu clase `Vector2D` original
2. **Extensibilidad**: Agregar métodos personalizados sin modificar el prototipo nativo
3. **Lazy initialization**: Inicialización diferida del vector interno hasta que p5.js esté disponible

La propiedad `_inicializarP5()` garantiza que el objeto `p5.Vector` interno se cree solo cuando sea necesario, resolviendo el problema de dependencias en el entorno JupyterLite.

---

In [ ]:
function Vector2D(x, y) {
    this._x = x || 0;
    this._y = y || 0;
}

Object.defineProperties(Vector2D.prototype, {
    x: {
        get: function() { return this._vec ? this._vec.x : this._x; },
        set: function(val) { 
            if (this._vec) this._vec.x = val;
            else this._x = val;
        }
    },
    y: {
        get: function() { return this._vec ? this._vec.y : this._y; },
        set: function(val) { 
            if (this._vec) this._vec.y = val;
            else this._y = val;
        }
    }
});

Vector2D.prototype._inicializarP5 = function() {
    if (!this._vec && typeof createVector === 'function') {
        this._vec = createVector(this._x, this._y);
    }
};

console.log("Vector2D base definido");

## 2. Variables Globales y Configuración del Sistema Físico

Esta celda establece el **estado global** del sistema de partículas y los parámetros físicos fundamentales de la simulación.

### Sistema de Partículas

Definimos un sistema de $n$ partículas (en este caso $n = 8$) donde cada partícula $i$ tiene asociado un vector de estado:

$$\vec{s}_i = \begin{pmatrix} \vec{r}_i \\ \vec{v}_i \\ \vec{a}_i \end{pmatrix} \in \mathbb{R}^6$$

donde:
- $\vec{r}_i = (x_i, y_i)$ es el **vector de posición**
- $\vec{v}_i = (v_{x_i}, v_{y_i})$ es el **vector de velocidad**
- $\vec{a}_i = (a_{x_i}, a_{y_i})$ es el **vector de aceleración**

### Campo Gravitatorio Uniforme

El modelo físico asume un campo gravitatorio uniforme:

$$\vec{g} = (0, g) \quad \text{con} \quad g > 0$$

donde la componente $y$ positiva indica dirección hacia abajo (sistema de coordenadas de pantalla).

### Coeficiente de Restitución

El parámetro $e \in [0, 1.2]$ representa el **coeficiente de restitución** que caracteriza las colisiones inelásticas según la relación de velocidades normal:

$$e = -\frac{v_{2n} - v_{1n}}{v_{2n}^0 - v_{1n}^0}$$

Para colisión con una superficie fija (suelo), se simplifica a:
$$v_{\text{después}} = -e \cdot v_{\text{antes}}$$

| Valor de $e$ | Tipo de colisión |
|-------------|------------------|
| $e = 0$ | Perfectamente inelástica (pegajosa) |
| $0 < e < 1$ | Inelástica (pérdida de energía) |
| $e = 1$ | Elástica (conservación de energía cinética) |
| $e > 1$ | Superelástica (ganancia de energía, no física realista) |

El slider interactivo permite explorar visualmente estos regímenes físicos.

---


In [ ]:
window.pelotas = [];
window.numPelotas = 8;
window.gravedad = null;
window.sueloY = null;
window.restitucion = null;
window.sliderRestitucion = null;

console.log("Variables globales inicializadas");

## CELDA 3 MARKDOWN
## 3. Implementación de la Física Vectorial y Dinámica de Partículas

Esta celda contiene la implementación completa de la **mecánica clásica newtoniana** usando operaciones vectoriales.

### Segunda Ley de Newton en Forma Vectorial

La ecuación fundamental de la dinámica:

$$\vec{F} = m\vec{a} \quad \Rightarrow \quad \vec{a} = \frac{\vec{F}}{m}$$

En forma diferencial para la simulación numérica:
$$\frac{d\vec{v}}{dt} = \vec{a}, \quad \frac{d\vec{r}}{dt} = \vec{v}$$

### Método de Integración de Euler

Utilizamos el **método de Euler explícito** (primera orden) para la integración numérica:

$$\vec{v}(t + \Delta t) = \vec{v}(t) + \vec{a}(t) \cdot \Delta t$$
$$\vec{r}(t + \Delta t) = \vec{r}(t) + \vec{v}(t) \cdot \Delta t$$

Con paso de tiempo $\Delta t = 1$ (un frame), las ecuaciones se simplifican a:
$$\vec{v}_{n+1} = \vec{v}_n + \vec{a}_n$$
$$\vec{r}_{n+1} = \vec{r}_n + \vec{v}_{n+1}$$

> **Nota numérica**: El método de Euler es condicionalmente estable y presenta error de truncamiento $O(\Delta t^2)$. Para simulaciones físicas más precisas se recomienda el método de **Euler-Cromer** (semi-implícito) o **Verlet**.

### Operaciones Vectoriales Implementadas

| Operación | Notación Matemática | Implementación |
|-----------|---------------------|----------------|
| Suma | $\vec{u} + \vec{v}$ | `incrementBy()` |
| Resta | $\vec{u} - \vec{v}$ | `decrementBy()` |
| Escalar por vector | $k\vec{v}$ | `scaleBy(k)` |
| Producto punto | $\vec{u} \cdot \vec{v} = u_x v_x + u_y v_y$ | `dotProduct()` |
| Magnitud | $\|\vec{v}\| = \sqrt{v_x^2 + v_y^2}$ | `length()` |

### Condiciones de Contorno (Colisiones)

Las colisiones con los bordes implementan:
1. **Corrección de posición**: Proyección sobre la frontera
2. **Inversión de velocidad normal**: $v_n \rightarrow -e \cdot v_n$
3. **Fricción tangencial**: $v_t \rightarrow 0.99 \cdot v_t$ (amortiguamiento)

### Clase Pelota: Encapsulamiento del Estado

Cada instancia de `Pelota` mantiene:
- **Propiedades cinemáticas**: $\vec{r}, \vec{v}, \vec{a}$ (objetos `Vector2D`)
- **Propiedades inerciales**: masa $m \propto r$ (radio)
- **Propiedades visuales**: color RGB, radio $r$

La masa se define proporcional al área: $m = k \cdot r$, asumiendo densidad superficial constante $\sigma$:
$$m = \sigma \cdot \pi r^2 \propto r \quad (\text{simplificado a } m = r/10)$$

---


In [ ]:
// ============ COMPLETAR VECTOR2D ============

Vector2D.prototype.lengthSquared = function() {
    this._inicializarP5();
    return this._vec.magSq();
};

Vector2D.prototype.length = function() {
    this._inicializarP5();
    return this._vec.mag();
};

Vector2D.prototype.clone = function() {
    this._inicializarP5();
    return new Vector2D(this._vec.x, this._vec.y);
};

Vector2D.prototype.add = function(vec) {
    this._inicializarP5();
    var other = vec._vec || vec;
    var result = this._vec.copy().add(other);
    return new Vector2D(result.x, result.y);
};

Vector2D.prototype.subtract = function(vec) {
    this._inicializarP5();
    var other = vec._vec || vec;
    var result = this._vec.copy().sub(other);
    return new Vector2D(result.x, result.y);
};

Vector2D.prototype.dotProduct = function(vec) {
    this._inicializarP5();
    var other = vec._vec || vec;
    return this._vec.dot(other);
};

Vector2D.prototype.negate = function() {
    this._inicializarP5();
    this._vec.mult(-1);
    return this;
};

Vector2D.prototype.normalize = function() {
    this._inicializarP5();
    this._vec.normalize();
    return this.length();
};

Vector2D.prototype.incrementBy = function(vec) {
    this._inicializarP5();
    var other = vec._vec || vec;
    this._vec.add(other);
    return this;
};

Vector2D.prototype.decrementBy = function(vec) {
    this._inicializarP5();
    var other = vec._vec || vec;
    this._vec.sub(other);
    return this;
};

Vector2D.prototype.scaleBy = function(k) {
    this._inicializarP5();
    this._vec.mult(k);
    return this;
};

Vector2D.distance = function(vec1, vec2) {
    vec1._inicializarP5();
    vec2._inicializarP5();
    return vec1._vec.dist(vec2._vec);
};

Vector2D.angleBetween = function(vec1, vec2) {
    vec1._inicializarP5();
    vec2._inicializarP5();
    return vec1._vec.angleBetween(vec2._vec);
};

// ============ CLASE PELOTA (CORREGIDA) ============

function Pelota(x, y, radio, col) {  // 'col' en lugar de 'color'
    this.pos = new Vector2D(x, y);
    this.vel = new Vector2D(random(-3, 3), random(-2, 2));
    this.acc = new Vector2D(0, 0);
    
    this.radio = radio || random(15, 30);
    // Usar color() de p5.js aquí, no guardar como propiedad llamada 'color'
    this.miColor = col || [random(100, 255), random(100, 255), random(100, 255)];
    this.masa = this.radio / 10;
}

Pelota.prototype.aplicarFuerza = function(fuerza) {
    var f = fuerza.clone().scaleBy(1 / this.masa);
    this.acc.incrementBy(f);
};

Pelota.prototype.actualizar = function() {
    this.vel.incrementBy(this.acc);
    this.pos.incrementBy(this.vel);
    this.acc = new Vector2D(0, 0);
};

Pelota.prototype.dibujar = function() {
    fill(this.miColor[0], this.miColor[1], this.miColor[2]);
    noStroke();
    ellipse(this.pos.x, this.pos.y, this.radio * 2);
    
    stroke(255, 0, 0, 150);
    strokeWeight(2);
    line(
        this.pos.x, this.pos.y,
        this.pos.x + this.vel.x * 3,
        this.pos.y + this.vel.y * 3
    );
};

Pelota.prototype.verificarBordes = function(suelo, e) {
    if (this.pos.y > suelo - this.radio) {
        this.pos.y = suelo - this.radio;
        this.vel.y *= -e;
        this.vel.x *= 0.99;
    }
    if (this.pos.y < this.radio) {
        this.pos.y = this.radio;
        this.vel.y *= -e;
    }
    if (this.pos.x < this.radio) {
        this.pos.x = this.radio;
        this.vel.x *= -e;
    }
    if (this.pos.x > width - this.radio) {
        this.pos.x = width - this.radio;
        this.vel.x *= -e;
    }
};

// ============ SETUP ============

function setup() {
    createCanvas(800, 500);
    
    window.gravedad = new Vector2D(0, 0.4);
    window.sueloY = 450;
    window.restitucion = 0.8;
    
    window.sliderRestitucion = createSlider(0.0, 1.2, 0.8, 0.05);
    window.sliderRestitucion.position(20, 20);
    window.sliderRestitucion.style('width', '200px');
    
    for (var i = 0; i < window.numPelotas; i++) {
        var x = random(50, width - 50);
        var y = random(50, 200);
        var pelota = new Pelota(x, y);
        window.pelotas.push(pelota);
    }
    
    console.log("Setup completado");
}

// ============ DRAW ============

function draw() {
    background(30);
    
    if (!window.pelotas || window.pelotas.length === 0) {
        fill(255);
        textAlign(CENTER, CENTER);
        textSize(20);
        text("Error: No hay pelotas", width/2, height/2);
        return;
    }
    
    window.restitucion = window.sliderRestitucion.value();
    var e = window.restitucion;
    
    stroke(100, 255, 100);
    strokeWeight(3);
    line(0, window.sueloY, width, window.sueloY);
    
    for (var i = 0; i < window.pelotas.length; i++) {
        var p = window.pelotas[i];
        
        var gravedadEscalada = window.gravedad.clone().scaleBy(p.masa);
        p.aplicarFuerza(gravedadEscalada);
        
        p.actualizar();
        p.verificarBordes(window.sueloY, e);
        p.dibujar();
    }
    
    fill(255);
    noStroke();
    textAlign(LEFT);
    textSize(14);
    text("PELOTAS CON VECTOR2D", 240, 30);
    text("Restitucion (e): " + e.toFixed(2), 240, 50);
    text("Pelotas: " + window.pelotas.length, 240, 70);
    
    textSize(12);
    fill(200);
    text("Ajusta el slider", 20, 470);
    
    var todasDetenidas = true;
    for (var i = 0; i < window.pelotas.length; i++) {
        if (Math.abs(window.pelotas[i].vel.y) > 0.5 || 
            window.pelotas[i].pos.y < window.sueloY - window.pelotas[i].radio - 5) {
            todasDetenidas = false;
            break;
        }
    }
    
    if (todasDetenidas) {
        for (var i = 0; i < window.pelotas.length; i++) {
            var p = window.pelotas[i];
            p.pos = new Vector2D(random(50, width - 50), random(50, 150));
            p.vel = new Vector2D(random(-5, 5), random(-2, 2));
        }
    }
}

console.log("Celda 3 lista");

## CELDA 4 MARKDOWN

## 4. Ejecución y Visualización: Representación Gráfica del Sistema Físico

Esta celda ejecuta el bucle principal de simulación mediante el comando mágico `%show` de JupyterLite, que inicializa el entorno p5.js.

### Ciclo de Renderizado

El sistema opera con dos funciones principales de p5.js:

1. **`setup()`**: Inicialización única
   - Creación del canvas: $\Omega = [0, 800] \times [0, 500] \subset \mathbb{R}^2$
   - Configuración de condiciones iniciales aleatorias
   - Instanciación de controles de interfaz (slider)

2. **`draw()`**: Bucle de animación (aproximadamente 60 FPS)
   - Paso de integración física
   - Detección y respuesta de colisiones
   - Renderizado gráfico

### Conservación de Energía (Análisis Teórico)

La energía mecánica total del sistema para la partícula $i$ es:

$$E_i = K_i + U_i = \frac{1}{2}m_i\|\vec{v}_i\|^2 + m_i g y_i$$

Para colisiones elásticas ($e = 1$), se conserva la energía cinética total. Para $e < 1$, la energía disipada en cada rebote es:

$$\Delta E = \frac{1}{2}m(v_{\text{antes}}^2 - v_{\text{después}}^2) = \frac{1}{2}mv_{\text{antes}}^2(1 - e^2)$$

### Reinicio Automático

El sistema detecta cuando todas las partículas alcanzan **equilibrio mecánico** (velocidad vertical $|v_y| < \varepsilon$ y contacto con el suelo), reiniciando la simulación con nuevas condiciones iniciales aleatorias:

$$\vec{r}_0 \in [50, 750] \times [50, 150], \quad \vec{v}_0 \in [-5, 5] \times [-2, 2]$$

### Interpretación Visual

- **Vectores rojos**: Representación gráfica de $\vec{v}$ (escalados 3× para visibilidad)
- **Colores**: Identificación única de cada partícula
- **Línea verde**: Frontera del suelo (condición de contorno $y = 450$)

Este sistema permite visualizar conceptos fundamentales de la mecánica clásica: **inercia**, **gravitación**, **colisiones** y **disipación de energía**, todo implementado con operaciones vectoriales rigurosas.

In [ ]:
%show